In [28]:
import openai
from pymongo import MongoClient
import chromadb
from chromadb.config import Settings
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings
import os


In [12]:
chroma_client = chromadb.PersistentClient(path="./chroma_db")
print(chroma_client.list_collections())


[Collection(name=rag_embeddings)]


In [25]:
openai_api_key = os.getenv("OPENAI_API_KEY")

# Conectar ao MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]

collection = chroma_client.get_collection("rag_embeddings")

# Inicializar embeddings
embedding_model = OpenAIEmbeddings(openai_api_key=openai_api_key)

In [26]:
# Função de pesquisa por query
def search_query(query, top_n=5):
    # Gerar o embedding da query
    query_embedding = embedding_model.embed_query(query)
    
    # Buscar no ChromaDB os embeddings mais próximos
    results = collection.query(
        query_embeddings=[query_embedding], 
        n_results=top_n
    )    
    
    # Recuperar os chunk_ids dos resultados
    chunk_ids = []
    metadatas = results["metadatas"][0]  # Acessando o primeiro item da lista de metadados
    
    # Iterando sobre os metadados para obter os chunk_ids
    for result in metadatas:
        chunk_ids.append(result["source_file"] + "_chunk_" + str(result["chunk_index"]))
    
    # Buscar os textos dos chunks no MongoDB
    chunks_text = []
    for chunk_id in chunk_ids:
        chunk = mongo_chunks.find_one({"chunk_id": chunk_id})
        if chunk:
            chunks_text.append(chunk["chunk_text"])
    
    # Retorna os textos dos chunks encontrados
    return "\n\n".join(chunks_text)



#exemplo de retorno da função
query = "ADA"
search_query(query)

'Afetadas (ADAs) e na Área de Influência Direta (AID) do empreendimento para desenvolver ações educativas, com ênfase nos afetados diretamente pelo empreendimento, minimizando os impactos ambientais e sociais, buscando uma atuação efetiva na melhoria da qualidade ambiental e de Para a sociedade civil da Área de Influência Direta (AID) e para os moradores  do bairro e colaboradores das áreas afetadas diretamente (ADA), a questão ambiental é bastante a)\nPlanejamento e Avaliação inicial\n\nPor outro lado, a cidade possui um grande potencial de desenvolvimento, especialmente no setor turístico, com a valorização da cultura local e a criação de novos atrativos.\nÁrea de Influência Direta (AID):\nÉ a região ao redor da ADA, que será afetada pelos impactos do empreendimento, como o aumento do Considerou-se um raio de 1000 metros.\nÁrea Diretamente Afetada (ADA):\nÉ construído e onde ocorrerão as remoção de vegetação ou a construção Área de Influência Indireta (AII):\n\nAfetada (ADA) é a área

In [29]:
#Função do chatbot com RAG
def chatbot(message: str):
    contexto = search_query(message)

    prompt = f"""
Você é um assistente especialista em licenciamento ambiental, com foco em auxiliar na elaboração de documentos EIA e RIMA. Use exclusivamente o contexto a seguir para responder de forma precisa e técnica.

Contexto extraído de documentos:
{contexto}

Pergunta do usuário:
{message}
"""
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "Você é um assistente técnico ambiental especializado em EIA e RIMA."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=1000
    )
    return response.choices[0].message.content

# Loop de interação
while True:
    user_input = input("Você: ")
    if user_input.lower() in ["sair", "exit", "quit"]:
        break
    print("Bot:", chatbot(user_input))

Você:  ola


Bot: Olá! Como posso ajudar você hoje?


Você:  sair
